In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden,1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 생성기와 판별기
# cost function : 확률적 함수
# Kullbak-Leibler divergence : GAN, VAE - 분포의 차를 확인하는 함수
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

def discriminator(inputs): # 실제의 분포를 확인
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### 최대우도 추정법
https://ratsgo.github.io/statistics/2017/09/23/MLE/

In [10]:
G = generator(Z) # 노이즈 => 이미지 생성
D_gene = discriminator(G) # 이미지를 판별 - 분포를 확인한다
D_real = discriminator(X) # 실제 이미지의 분포를 확인
# log 확률값 => 정보량
# 확률이 높아지면 정보량이 작아지고, 확률이 낮아지면 정보량이 커짐 (tfidf 논리)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene)) # 최대 우도 추정법 - 적합한 분포가 될 수 있도록
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]
# minimize : 확률이 높은 것이 유리하므로 -를 붙여서 최소화
# var_list : 역전파 변수를 지정해 줄 수 있음
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [11]:
%matplotlib inline
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [13]:
# 이미지 분포의 특징을 추출하는 망
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    if epoch == 0 or (epoch+1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z:noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)
print('최적화 완료!')

Epoch: 0000 D loss: -0.4085 G loss: -2.207
Epoch: 0001 D loss: -0.1881 G loss: -2.557
Epoch: 0002 D loss: -0.6409 G loss: -1.327
Epoch: 0003 D loss: -0.3949 G loss: -1.8
Epoch: 0004 D loss: -0.3156 G loss: -2.082
Epoch: 0005 D loss: -0.2667 G loss: -2.466
Epoch: 0006 D loss: -0.4115 G loss: -2.258
Epoch: 0007 D loss: -0.3092 G loss: -2.616
Epoch: 0008 D loss: -0.5361 G loss: -1.977
Epoch: 0009 D loss: -0.3383 G loss: -2.323
Epoch: 0010 D loss: -0.3157 G loss: -2.293
Epoch: 0011 D loss: -0.3892 G loss: -2.256
Epoch: 0012 D loss: -0.2721 G loss: -2.448
Epoch: 0013 D loss: -0.4596 G loss: -2.298
Epoch: 0014 D loss: -0.498 G loss: -2.001
Epoch: 0015 D loss: -0.3258 G loss: -2.311
Epoch: 0016 D loss: -0.4872 G loss: -2.177
Epoch: 0017 D loss: -0.5873 G loss: -2.095
Epoch: 0018 D loss: -0.491 G loss: -2.128
Epoch: 0019 D loss: -0.3575 G loss: -2.764
Epoch: 0020 D loss: -0.4988 G loss: -2.165
Epoch: 0021 D loss: -0.5392 G loss: -2.125
Epoch: 0022 D loss: -0.3649 G loss: -2.634
Epoch: 0023 D l

GAN 기본 개념 및 다양한 활용
https://pathmind.com/kr/wiki/generative-adversarial-network-gan